# Gesture Recognition
In this project, we are building a 3D Conv model that will be able to predict the 5 gestures correctly. Import the following libraries to get started.

In [1]:
import numpy as np
import os
from PIL import Image
import datetime
import os
import matplotlib.pyplot as plt
import warnings  
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)


Using TensorFlow backend.


In this block, we read the folder names for training and validation. We also set the `batch_size` here. Note that we set the batch size in such a way that we are able to use the GPU in full capacity. We keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. We have to experiment with `img_idx`, `y`,`z` and normalization such that we get high accuracy.

In [4]:
images=[i for i in range(0,30,2)]
images

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]

In [5]:
img_width=220
img_height=220
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = t.size//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_width,img_height,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    res_image=image.resize((img_width,img_height))
                    res_image_arr=np.asarray(res_image)
                    res_image_arr=res_image_arr/255
                    batch_data[folder,idx,:,:,0] = res_image_arr[:,:,0]
                    batch_data[folder,idx,:,:,1] = res_image_arr[:,:,1]
                    batch_data[folder,idx,:,:,2] = res_image_arr[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        rem = len(t)%batch_size
        for folder in range(rem): #iterate over remaining folders
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = Image.open(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                res_image=image.resize((img_width,img_height))
                res_image_arr=np.asarray(res_image)
                res_image_arr=res_image_arr/255
                batch_data[folder,idx,:,:,0] = res_image_arr[:,:,0]
                batch_data[folder,idx,:,:,1] = res_image_arr[:,:,1]
                batch_data[folder,idx,:,:,2] = res_image_arr[:,:,2]            
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40
print ('# epochs =', num_epochs)
batch_size = 15

# training sequences = 663
# validation sequences = 100
# epochs = 40


## Model
Here we make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. We would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,LSTM,BatchNormalization
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D, AveragePooling2D,AveragePooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import *
#####Conv3D
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu',input_shape=(len(images), img_width, img_height, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

Now that we have written the model, the next step is to `compile` the model. When we print the `summary` of the model, we'll see the total number of parameters you have to train.

In [8]:
optimiser=optimizers.Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 218, 218, 32)  2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 109, 109, 32)   0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 4, 107, 107, 64)   55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 2, 53, 53, 64)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 359552)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1797765   
Total params: 1,855,749
Trainable params: 1,855,749
Non-trainable params: 0
_________________________________________________________________


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.25,
                              patience=1, min_lr=0.0001) # the REducelronplateau code 
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the model at the end of each epoch.

## Conv3D model

In [12]:
history1=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 15
Source path =  ./Project_data/train ; batch size = 15
Epoch 1/40
45/45 [==============================] - 58s 1s/step - loss: 2.3901 - categorical_accuracy: 0.2622 - val_loss: 1.3694 - val_categorical_accuracy: 0.3333

Epoch 00001: val_loss improved from inf to 1.36940, saving model to model_init_2020-05-0416_54_17.420531/model-00001-2.39010-0.26222-1.36940-0.33333.h5
Epoch 2/40
45/45 [==============================] - 49s 1s/step - loss: 1.1899 - categorical_accuracy: 0.5022 - val_loss: 1.1113 - val_categorical_accuracy: 0.6095

Epoch 00002: val_loss improved from 1.36940 to 1.11134, saving model to model_init_2020-05-0416_54_17.420531/model-00002-1.18994-0.50222-1.11134-0.60952.h5
Epoch 3/40
45/45 [==============================] - 49s 1s/step - loss: 0.8622 - categorical_accuracy: 0.7022 - val_loss: 1.1880 - val_categorical_accuracy: 0.5143

Epoch 00003: val_loss did not improve from 1.11134
Epoch 4/40
45/45 [======================

## Conv2D RNN model

In [12]:
# load model
import gc
print(gc.collect())

vision_model=Sequential()
vision_model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu'),input_shape=(len(images), img_width, img_height, 3)))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Conv2D(128, kernel_size=(3, 3), activation='relu')))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Conv2D(256, kernel_size=(3, 3), activation='relu')))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Conv2D(512, kernel_size=(3, 3), activation='relu')))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Conv2D(1024, kernel_size=(3, 3), activation='relu')))
vision_model.add(TimeDistributed(BatchNormalization()))
vision_model.add(TimeDistributed(AveragePooling2D(pool_size=(2, 2))))
vision_model.add(TimeDistributed(Flatten()))
vision_model.add(GRU(150))
vision_model.add(Dense(output_dim=5, activation='softmax'))


22


In [13]:
vision_model.compile(optimizer=optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['categorical_accuracy'])
print(vision_model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 218, 218, 32)  896       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 218, 218, 32)  128       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 109, 109, 32)  0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 107, 107, 64)  18496     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 107, 107, 64)  256       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 53, 53, 64)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 15, 51, 51, 128)  

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [18]:
history=vision_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 15
 Epoch 1/40
./Project_data/val ; batch size = 15
45/45 [==============================] - 187s 4s/step - loss: 1.5575 - categorical_accuracy: 0.4593 - val_loss: 1.8165 - val_categorical_accuracy: 0.3429

Epoch 00001: val_loss improved from inf to 1.81648, saving model to model_init_2020-05-0415_58_24.094028/model-00001-1.55752-0.45926-1.81648-0.34286.h5
Epoch 2/40
45/45 [==============================] - 73s 2s/step - loss: 0.9328 - categorical_accuracy: 0.6385 - val_loss: 1.7308 - val_categorical_accuracy: 0.4857

Epoch 00002: val_loss improved from 1.81648 to 1.73080, saving model to model_init_2020-05-0415_58_24.094028/model-00002-0.93277-0.63852-1.73080-0.48571.h5
Epoch 3/40
45/45 [==============================] - 72s 2s/step - loss: 0.7199 - categorical_accuracy: 0.7141 - val_loss: 1.0203 - val_categorical_accuracy: 0.6000

Epoch 00003: val_loss improved from 1.73080 to 1.02027, saving model to model_init_2020-05